### Imports

In [1]:
from definitions import *

### Make quiz prompt

In [2]:
with open("../../metrics/knowledge_tests/obrazovaka/oblomov.json", "r") as f:
    quiz = json.load(f)

In [3]:
part = quiz[0]
part

{'question': 'К какому литературному направлению относится роман Гончарова «Обломов»?',
 'answers': [{'answer': 'Романтизм;', 'is_correct': 0},
  {'answer': 'Реализм;', 'is_correct': 1},
  {'answer': 'Классицизм;', 'is_correct': 0},
  {'answer': 'Сентиментализм.', 'is_correct': 0}]}

In [4]:
def build_prompt(quiz_part):
    processed_answers = ""
    for i, answer in enumerate(quiz_part["answers"]):
        processed_answers += f"{i + 1}. {answer['answer']}\n"
    # return f"Выбери один верный вариант из предложенных.\nВопрос: {quiz_part['question']}\nВарианты ответа:\n{processed_answers}"
    return f"Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.\nВопрос: {quiz_part['question']}\nВарианты ответа:\n{processed_answers}"

In [5]:
print(build_prompt(part))

Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.
Вопрос: К какому литературному направлению относится роман Гончарова «Обломов»?
Варианты ответа:
1. Романтизм;
2. Реализм;
3. Классицизм;
4. Сентиментализм.



### Learn to make requests to ollama

In [6]:
! ollama list

NAME         	ID          	SIZE  	MODIFIED    
llama2:latest	78e26419b446	3.8 GB	12 days ago	


In [13]:
def get_model_response(model, prompt):
    headers = {
        'Content-Type': 'application/json',
    }
    json_data = {
        'model': model,
        'prompt': prompt,
        'stream': False,
        "options": {
            "seed": 123,
            "temperature": 0
        }
    }

    response = requests.post('http://localhost:11434/api/generate', headers=headers, json=json_data)
    payload = json.loads(response.content)
    return payload["response"]

In [15]:
get_model_response("llama2", "say hi")

' Unterscheidung between "hi" and "hello"\n\n"Hi" is a casual greeting that is often used in informal settings, such as with friends or in casual conversations. It is a more abbreviated version of the traditional greeting "hello."\n\nOn the other hand, "hello" is a more formal and polite greeting that is commonly used in professional or formal settings. It is also used to greet someone you don\'t know well or as a way of acknowledging someone\'s presence.\n\nIn general, "hi" is a more casual and informal greeting, while "hello" is more formal and polite. However, both greetings are widely accepted and can be used in different contexts depending on the situation and the relationship between the people involved.'

### Try to pass quiz with a model

In [16]:
prompt = build_prompt(part)
print(prompt)
get_model_response(model, prompt)

Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.
Вопрос: В чем автор видит причины «обломовщины»?
Варианты ответа:
1. В деградации и застое чистой, нерасчетливой, нежной, но ленивой души героя;
2. В окружении героя, которое не давало ему развиваться;
3. В недостаточно хорошем образовании героя;
4. В отсутствии достойных стимулов к развитию.



'1.'

In [17]:
len(quiz)

19

In [29]:
def eval_model_on_quiz(model, quiz):
    num_ok = 0
    num_all = 0
    for part in tqdm(quiz):
        prompt = build_prompt(part)
        answer = get_model_response(model, prompt)
        first_part = answer.split()[0]
        if first_part.endswith("."):
            first_part = first_part[:-1]
        ok_answer = -1
        for i, x in enumerate(part["answers"]):
            if x["is_correct"] == 1:
                ok_answer = i + 1
        try:
            first_part = int(first_part)
        except Exception:
            print(f"Cannot convert model first part answer to int: '{answer}'")
        if ok_answer == first_part:
            num_ok += 1
        num_all += 1
    print(f"result is {num_ok} out of {num_all}. {num_ok / num_all * 100:.2f}%")
    return num_ok, num_all

In [36]:
model = "llama2"
tests_dir = Path("../../metrics/knowledge_tests/obrazovaka/")
rows = []
for test_path in tests_dir.glob("*"):
    with open(test_path, "r") as f:
        quiz = json.load(f)
    num_ok, num_all = eval_model_on_quiz(model, quiz)
    rows.append({
        "model": model,
        "test_path": test_path,
        "num_ok": num_ok,
        "num_all": num_all,
        "percent": float(f"{(num_ok / num_all * 100):.2f}"),
    })

SyntaxError: invalid decimal literal (1891919668.py, line 13)

In [43]:
for row in rows:
    row["percent"] = float(f'{(row["num_ok"] / row["num_all"] * 100):.2f}')

In [44]:
df = pd.DataFrame(rows)

In [45]:
df

,model,test_path,num_ok,num_all,percent
0,llama2,../../metrics/knowledge_tests/obrazovaka/the_d...,3,16,18.75
1,llama2,../../metrics/knowledge_tests/obrazovaka/the_b...,1,10,10.00
2,llama2,../../metrics/knowledge_tests/obrazovaka/and_q...,6,18,33.33
3,llama2,../../metrics/knowledge_tests/obrazovaka/oblom...,7,19,36.84
4,llama2,../../metrics/knowledge_tests/obrazovaka/eveni...,1,10,10.00
5,llama2,../../metrics/knowledge_tests/obrazovaka/docto...,3,13,23.08
6,llama2,../../metrics/knowledge_tests/obrazovaka/war_a...,17,70,24.29
7,llama2,../../metrics/knowledge_tests/obrazovaka/the_m...,4,16,25.00
8,llama2,../../metrics/knowledge_tests/obrazovaka/dead_...,4,19,21.05
9,llama2,../../metrics/knowledge_tests/obrazovaka/the_g...,2,10,20.00


In [46]:
df.to_csv("../../metrics/knowledge_tests/obrazovaka/full_results.csv", index=False)